In [ ]:
# !conda list dolfin
# !conda list mshr

In [ ]:
from dolfin import *
import numpy as np
# import pygad
import pyswarms as ps
from pyswarms.backend.handlers import OptionsHandler
import matplotlib.pyplot as plt
import mshr as mr

set_log_level(50)

In [ ]:
# 创建网格
rectangle = mr.Rectangle(Point(0.0, 0.0), Point(9.0, 4.0))
hole_1 = mr.Circle(Point(3.0, 2.0), 1)
hole_2 = mr.Circle(Point(6.2, 2.0), 1)
domain = rectangle - hole_1 - hole_2

# 创建网格
mymesh = mr.generate_mesh(domain, 16)
vertex_num = mymesh.num_vertices()
cell_num = mymesh.num_cells()
plot(mymesh)
print(f"vertex number: {vertex_num}")
print(f"cell number: {cell_num}")

In [ ]:
# 初始化
nelx, nely = 10, 10
# 二维正交各向异性
E1, E2, mu12, G12 = 130000, 7700, 0.33, 4800
# 转动角度
Theta = 0
Theta_init = 5
# 载荷
forcing = Constant((500, 0))

# 弹性张量
C2D_Iso = np.array([
    [E1/(1-mu12**2),        mu12*E2/(1-mu12**2),         0  ],
    [mu12*E2/(1-mu12**2),   E2/(1-mu12**2),              0  ],
    [0,                     0,                           G12]
])

# 转轴公式
def T2D_inv(theta):
    theta = theta * np.pi / 180.0
    c = cos(theta)
    s = sin(theta)
    Trans = np.array([
        [c**2,  s**2,   -2*s*c],
        [s**2,  c**2,   2*s*c],
        [s*c,   -s*c,   c**2 - s**2]
    ])
    return Trans

# 准备网格
mesh = UnitSquareMesh(nelx, nely)
V = VectorFunctionSpace(mymesh, "CG", 1)
T = FunctionSpace(mymesh, "DG", 0)
u_sol = Function(V)
Theta_sol = Function(T)
u_trial = TrialFunction(V)
v_test = TestFunction(V)

In [ ]:
# 定义物理方程
# 物理方程
def epsilon(u):
    engineering_strain = 0.5 * (nabla_grad(u) + nabla_grad(u).T)
    return engineering_strain

# 本构方程
def sigma_tensor(u, Theta_Ev):
    # 计算应变张量
    epsilon_ij = epsilon(u)
    ep = as_vector([epsilon_ij[0,0], epsilon_ij[1,1], epsilon_ij[0,1]])
    # 使用弹性系数计算应力张量
    Q_bar = np.dot(np.dot(T2D_inv(Theta_Ev), C2D_Iso), T2D_inv(Theta_Ev).T)
    sigma_ij = np.dot(Q_bar, ep)

    return as_tensor([[sigma_ij[0], sigma_ij[2]],
                      [sigma_ij[2], sigma_ij[1]]])

# 弹性能密度
def psi(u, Theta):
    return 0.5 * inner(sigma_tensor(u, Theta), epsilon(u))

In [ ]:
# 定义边界
# 狄利克雷边界
def clamped_boundary(x, on_boundary):
    return on_boundary and x[0] < DOLFIN_EPS

bc = DirichletBC(V, Constant((0.0, 0.0)), clamped_boundary)

# 自然边界
class RightEnd(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 9) < DOLFIN_EPS and (abs(x[1] - 2) < 2)
right_end_boundary = RightEnd()

class TopEnd(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 0.5) < 0.5 and (abs(x[1] - 1) < DOLFIN_EPS)
top_end_boundary = TopEnd()

boundary_mark = MeshFunction("size_t", mymesh, mymesh.topology().dim()-1)
boundary_mark.set_all(0)
right_end_boundary.mark(boundary_mark, 1)
top_end_boundary.mark(boundary_mark, 2)

In [ ]:
# 有限元求解
def FEA(V:VectorFunctionSpace, lhs, rhs, u:Function, bc:DirichletBC) -> Function: 
    problem = LinearVariationalProblem(lhs, rhs, u, bc)
    solver = LinearVariationalSolver(problem)
    return solver.solve()

In [ ]:
# 可视化
def plot_theta_vector(theta:Function, mesh, scale=30, width=0.001):
    theta_rad = np.deg2rad(Theta_sol.vector()[:])  # 将角度转换为弧度
    x_values = np.cos(theta_rad)
    y_values = np.sin(theta_rad)
    # 将二维向量表示合并为一个二维数组
    vector_values = np.vstack((x_values, y_values)).T  # 转置使每行是一个向量
    # 获取网格的中心点
    cell_midpoints = [cell.midpoint() for cell in cells(mesh)]
    # 提取中心点的坐标
    x_coords = np.array([p.x() for p in cell_midpoints])
    y_coords = np.array([p.y() for p in cell_midpoints])
    u = vector_values[:, 0]
    v = vector_values[:, 1]
    # 可视化向量场
    plt.figure()
    # plot(mesh)
    # plt.quiver(x_coords, y_coords, u, v)
    # 绘制正方向的箭头
    plt.quiver(x_coords, y_coords, u, v, angles='xy', scale_units='xy', scale=scale, width=width, color='b')
    # 绘制反方向的箭头
    # plt.quiver(x_coords, y_coords, -u, -v, angles='xy', scale_units='xy', scale=scale, width=width, color='b')

    plt.title("Vector field")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.axis("equal")
    plt.show()

In [ ]:
# 定义优化问题
def minimize_strain_energy(x):
    theta_values = x[:, :T.dim()]
    total_energy = []
    
    for i in range(theta_values.shape[0]):
        theta_func = Function(T)
        theta_func.vector()[:] = theta_values[i, :]
        Theta_sol.assign(theta_func)
        
        lhs = inner(sigma_tensor(u_trial, Theta_sol), nabla_grad(v_test)) * dx
        rhs = dot(forcing, v_test) * ds(subdomain_data=boundary_mark, domain=mymesh, subdomain_id=1)
        FEA(V, lhs, rhs, u_sol, bc)
        
        # 计算单元应变能密度
        psi_density = project(psi(u_sol, Theta_sol), T)
        cell_energies = psi_density.vector()[:]
        
        # 计算总应变能
        total_energy_density = np.sum(cell_energies)
        
        # 计算最大和最小应变能
        max_energy_density = np.max(cell_energies)
        min_energy_density = np.min(cell_energies)
        
        # 确保最小应变能不为零
        if min_energy_density == 0:
            min_energy_density = 1e-10
        
        # 计算最大最小应变能之比
        energy_ratio = max_energy_density / min_energy_density
        
        # 目标函数包括总应变能和能量比
        objective = total_energy_density + 10000 * energy_ratio
        total_energy.append(objective)
    
    return np.array(total_energy)

# 设置PSO超参数
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}  # 初始参数
oh_strategy = {'w': 'exp_decay', 'c1': 'lin_variation'}  # 动态调整策略

# 创建PSO优化器实例
optimizer = ps.single.GlobalBestPSO(n_particles=200, dimensions=T.dim(), options=options, oh_strategy=oh_strategy)

# 初始化粒子群的最佳位置和成本
optimizer.swarm.pbest_cost = np.inf * np.ones(optimizer.swarm_size[0])
optimizer.swarm.pbest_pos = optimizer.swarm.position.copy()

# 初始评估粒子群，计算pbest和gbest
optimizer.swarm.current_cost = minimize_strain_energy(optimizer.swarm.position)
optimizer.swarm.pbest_cost = optimizer.swarm.current_cost.copy()
optimizer.swarm.pbest_pos = optimizer.swarm.position.copy()
optimizer.swarm.best_cost = np.min(optimizer.swarm.current_cost)
optimizer.swarm.best_pos = optimizer.swarm.position[np.argmin(optimizer.swarm.current_cost)].copy()

# 定义速度和位置的约束参数
clamp = None  # 无速度限制
max_bound = 90 * np.ones(T.dim())
min_bound = -90 * np.ones(T.dim())
bounds = (min_bound, max_bound)  # 无位置限制

# 使用默认的边界处理函数
velocity_handler = ps.backend.handlers.VelocityHandler(strategy="unmodified")
boundary_handler = ps.backend.handlers.BoundaryHandler(strategy="periodic")

# 创建 OptionsHandler 实例
oh = OptionsHandler(strategy=oh_strategy)

# 执行优化
iterations = 2000
for i in range(iterations):
    # 更新PSO参数
    optimizer.swarm.options = oh(optimizer.swarm.options, iternow=i, itermax=iterations)
    
    # 执行单次迭代
    optimizer.swarm.velocity = ps.backend.operators.compute_velocity(optimizer.swarm, clamp=clamp, vh=velocity_handler, bounds=bounds)
    optimizer.swarm.position = ps.backend.operators.compute_position(optimizer.swarm, bounds=bounds, bh=boundary_handler)
    optimizer.swarm.current_cost = minimize_strain_energy(optimizer.swarm.position)
    optimizer.swarm.pbest_pos, optimizer.swarm.pbest_cost = ps.backend.operators.compute_pbest(optimizer.swarm)
    gbest_cost = np.min(optimizer.swarm.pbest_cost)
    if gbest_cost < optimizer.swarm.best_cost:
        optimizer.swarm.best_cost = gbest_cost
        optimizer.swarm.best_pos = optimizer.swarm.pbest_pos[np.argmin(optimizer.swarm.pbest_cost)].copy()

# 获取优化结果
cost = optimizer.swarm.best_cost
pos = optimizer.swarm.best_pos
print(f"Best cost: {cost}")
print(f"Best position: {pos}")

In [ ]:
# 可视化
import matplotlib.pyplot as plt
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)

plot_cost_history(cost_history=optimizer.cost_history)
plt.show()

Theta_sol.vector()[:] = pos
plot_theta_vector(Theta_sol, mesh=mymesh, scale=5, width=0.002)

plt.show()
plot(Theta_sol)
a = Theta_sol.vector()[:]
print(a.max()/a.min())

In [ ]:
# 计算应变能密度
psi_density = project(psi(u_sol, Theta_sol), T)

# 获取每个单元的中心点
cell_midpoints = [cell.midpoint() for cell in cells(mymesh)]
x_coords = np.array([p.x() for p in cell_midpoints])
y_coords = np.array([p.y() for p in cell_midpoints])

# 获取每个单元的应变能密度值
psi_values = psi_density.vector()[:]

# 可视化应变能密度并在每个单元显示数值
plt.figure()
plot(psi_density, title="Strain Energy Density")
# plt.colorbar()
print("ratiao: " + str(psi_values.max() / psi_values.min()))

for i, (x, y) in enumerate(zip(x_coords, y_coords)):
    plt.text(x, y, f"{psi_values[i]:.2f}", color="black", ha="center", va="center", fontsize=8)

plt.show()

In [ ]:
plot(mymesh)

In [ ]:
# 计算流线图
from scipy.interpolate import griddata
def plot_streamlines(theta_func, mesh, scale=1, density=1):
    theta_rad = np.deg2rad(theta_func.vector()[:])  # 将角度转换为弧度
    x_values = np.cos(theta_rad)
    y_values = np.sin(theta_rad)
    
    # 将二维向量表示合并为一个二维数组
    vector_values = np.vstack((x_values, y_values)).T  # 转置使每行是一个向量
    
    # 获取网格的中心点
    cell_midpoints = [cell.midpoint() for cell in cells(mesh)]
    
    # 提取中心点的坐标
    x_coords = np.array([p.x() for p in cell_midpoints])
    y_coords = np.array([p.y() for p in cell_midpoints])
    
    # 创建网格
    xi = np.linspace(x_coords.min(), x_coords.max(), 100)
    yi = np.linspace(y_coords.min(), y_coords.max(), 100)
    Xi, Yi = np.meshgrid(xi, yi)
    
    # 插值矢量场
    Ui = griddata((x_coords, y_coords), vector_values[:, 0], (Xi, Yi), method='cubic')
    Vi = griddata((x_coords, y_coords), vector_values[:, 1], (Xi, Yi), method='cubic')
    
    # 创建有效区域的掩膜
    mask = griddata((x_coords, y_coords), np.ones_like(x_coords), (Xi, Yi), method='nearest', fill_value=0)
    
    # 使用掩膜来过滤掉无效区域
    Ui = np.ma.array(Ui, mask=(mask == 0))
    Vi = np.ma.array(Vi, mask=(mask == 0))
    
    # 绘制流线图
    plt.figure(figsize=(10, 5))
    
    ax = plt.subplot(1, 1, 1)
    strm = ax.streamplot(Xi, Yi, Ui, Vi, density=density, color='r')
    ax.set_title("Streamlines with Masking")
    
    # 将掩膜转换为布尔数组
    bool_mask = mask.astype(bool)
    
    # 显示掩膜区域
    ax.imshow(~bool_mask, extent=(x_coords.min(), x_coords.max(), y_coords.min(), y_coords.max()), 
              alpha=0.5, cmap='gray', aspect='auto')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 可视化流线图
plot_streamlines(Theta_sol, mesh=mymesh, scale=5, density=1.5)